# Product Neural Network

PNN 是在 DeepCrossing 模型上的修改，在 DeepCrossing 模型中，每一个 field 经过 Embedding 过后直接被拼接在了一起，然后送入全连接层进行后面的计算。在 PNN 模型中，在第一个全连接层之前增加了一个交叉乘积层，交叉的内容是 Embedding 之后的任意两个 field。PNN 的另一个修改是把 DeepCrossing 中的残差 Block 替换成了全连接层。在 PNN 中的交叉采用的是向量的外积，计算公式为 $z = |x||y| sin(x, y)$

In [2]:
# build field dict

import os

BASEDIR = os.getcwd()

fields_dict = {}

with open(BASEDIR + '/assets/datasets/criteo_ctr/small_train.txt') as f:
    line = f.readline()
    line = line.strip('\n')

    while line:
        elems = line.split(' ')

        for i in range(1, len(elems)):
            field, feature, value = elems[i].split(':')
            if field not in fields_dict:
                fields_dict[field] = {}
                fields_dict[field]['field'] = {feature: 0}
                fields_dict[field]['last_idx'] = 0
            elif feature not in fields_dict[field]['field']:
                next_idx = fields_dict[field]['last_idx'] + 1
                fields_dict[field]['field'][feature] = next_idx
                fields_dict[field]['last_idx'] = next_idx
 
        line = f.readline()
        line = line.strip('\n')

for field in fields_dict.keys():
    next_idx = fields_dict[field]['last_idx'] + 1
    fields_dict[field]['field']['none'] = next_idx
    fields_dict[field]['last_idx'] = next_idx

field_cnt = len(fields_dict)
field_cnt

18

In [ ]:
# build train data

X_train = []
y_train = []
idx = 0
with open(BASEDIR + '/assets/datasets/criteo_ctr/small_train.txt') as f:
    line = f.readline()
    line = line.strip('\n')

    while line:
        elems = line.split(' ')
        y_train.append(int(elems[0]))

        X_train.append([])
        for i in range(field_cnt):
            X_train[idx].append([])

        for i in range(1, len(elems)):
            field, feature, value = elems[i].split(':')
            field_idx = int(field)
            X_train[idx][field_idx].append(fields_dict[field]['field'][feature])

        for i in range(field_cnt):
            if len(X_train[idx][i]) == 0:
                X_train[idx][i].append(fields_dict[str(i)]['field']['none'])
       
        line = f.readline()
        line = line.strip('\n')
        idx += 1


In [ ]:
# build Product Neural Network

import torch
import torch.nn as nn
import torch.nn.functional as F

import numpy as np


class CrossLayer(nn.Module):

    def __init__(self, INPUT_DIMENSION, OUTPUT_DIMENSION):
        super(CrossLayer, self).__init__()
        self.layer = nn.Linear(INPUT_DIMENSION, OUTPUT_DIMENSION, bias=True).double()

    def forward(self, input_features1, input_features2):
        x = torch.cross(input_features1, input_features2)
        return self.layer(x)


class ProductNeuralNetwork(nn.Module):
    def __init__(self, fields_dict, embedding_size, z_layers, p_layers, fc_layers):
        super(ProductNeuralNetwork, self).__init__()

        self.fields_dict = fields_dict
        self.input_embeddings = []
        self.embedding_size = embedding_size
        self.fc_layers = fc_layers
        self.z_layers = z_layers
        self.p_layers = p_layers

        for j in range(len(self.fields_dict)):
            self.input_embeddings.append(
                nn.Linear(len(self.fields_dict[str(j)]['field']), self.embedding_size).double())

        self.z_layers_seq = []
        self.p_layers_seq = []

        for z_layer in self.z_layers:
            self.z_layers_seq.append(nn.Linear(self.embedding_size + 1, z_layer).double())

        for p_layer in self.p_layers:
            self.p_layers_seq.append(CrossLayer(self.embedding_size, p_layer))

        z_layers_output_num = sum(self.z_layers)
        p_layers_output_num = sum(self.p_layers)

        fc_layers.append(nn.Linear(z_layers_output_num + p_layers_output_num, fc_layers[0]).double())
        self.fc_layers_seq = []
        for i in range(1, len(self.fc_layers)):
            self.fc_layers_seq.append(nn.Linear(self.fc_layers[i - 1], self.fc_layers[i]).double())

        self.module = nn.Sequential(fc_layers)

    def forward(self, x):
        embedding_inputs = []
        batch_size = len(x)
        for idx in range(len(self.input_embeddings)):
            fields_size = len(self.fields_dict[str(idx)]['field'])
            fields_idx_input = torch.zeros(fields_size, batch_size, dtype=dtype, device=device)
            for i in range(batch_size):
                for q in x[i][idx]:
                    fields_idx_input[q, i] = 1.0
            t = self.input_embeddings[idx](fields_idx_input.T)
            embedding_inputs.append(t)

        # z part
        z_outputs = []
        for embedding_input, z_layer in zip(embedding_inputs, self.z_layers_seq):
            z_outputs.append(z_layer(embedding_input))

        # p part
        p_outputs = []
        for i in range(len(embedding_inputs) - 1):
            for j in range(i + 1, len(embedding_inputs)):
                p_outputs.append(
                    self.p_layers_seq[i * len(embedding_inputs) + j](embedding_inputs[i], embedding_inputs[j]))

        fc_input = torch.cat(z_outputs + p_outputs, 1)
        out = self.fc_layers_seq[0](fc_input)
        
        for i in range(1, len(self.fc_layers)):
            out = self.fc_layers_seq[i](out)
            
        return out


In [1]:
# PyTorch Version

import torch.optim as optim

dtype = torch.double
device = torch.device('cpu')

LEARNING_RATE = 1e-3

EPOCH = 10
PRINT_STEP = EPOCH / 10
N = len(y_train)

z_layers = [5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5]
p_layers = [5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5] 
fc_layers = [5, 5, 1]

pnn = ProductNeuralNetwork(fields_dict, 8)

BATCH_SIZE = 8
loss_fn = nn.BCELoss(size_average=True, reduce=True)
optimizer = optim.Adam(pnn.parameters(), lr=LEARNING_RATE)

for epoch in range(EPOCH):
    start = 0
    end = start + BATCH_SIZE

    while start < N:
        optimizer.zero_grad()
        if end >= N:
            end = N

        X_batch = X_train[start:end]
        y_batch = torch.from_numpy(np.array(y_train[start:end], np.float)).reshape(-1, BATCH_SIZE)

        y_hat = pnn(X_batch).reshape(-1, BATCH_SIZE)
        loss = loss_fn(y_hat, y_batch)

        loss.backward()
        optimizer.step()

        start = end
        end = start + BATCH_SIZE

    if epoch % PRINT_STEP == 0:
        print('EPOCH: %d, loss: %f' % (epoch, loss))

NameError: name 'torch' is not defined